In [ ]:
import os

from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import xarray as xr

from sdm_eurec4a.visulization import set_custom_rcParams, symlog_from_array
from sdm_eurec4a.identifications import (
    select_individual_cloud_by_id,
    match_clouds_and_cloudcomposite,
    match_clouds_and_dropsondes,
)
from sdm_eurec4a.reductions import x_y_flatten
from sdm_eurec4a.pySD import probdists

from sdm_eurec4a.conversions import msd_from_psd

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
from matplotlib import rc

# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
# rc('font',**{'family':'serif','serif':['Palatino']})
rc("text", usetex=False)

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = os.path.abspath("/home/m/m301096/repositories/sdm-eurec4a/scripts/CLEO/initalize")
print(script_dir)

REPOSITORY_ROOT = Path(script_dir).parents[2]
print(REPOSITORY_ROOT)

fig_path = REPOSITORY_ROOT / Path("results/CLEO/initilization/fitting_psd")
fig_path.mkdir(parents=True, exist_ok=True)

/home/m/m301096/repositories/sdm-eurec4a/scripts/CLEO/initalize
/home/m/m301096/repositories/sdm-eurec4a


### Load datasets

In [ ]:
# Load data
# mask_name = "cloud_mask"
# chosen_id = 1421

mask_name = "rain_mask"
chosen_id = 77

# chosen_id = None

subfig_path = fig_path / Path(f"{mask_name}_{chosen_id}")
subfig_path.mkdir(parents=True, exist_ok=True)

identified_clouds = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(
        f"data/observation/cloud_composite/processed/identified_clouds/identified_clouds_{mask_name}.nc"
    )
)
# select only clouds which are between 800 and 1100 m
identified_clouds = identified_clouds.where(
    (identified_clouds.alt >= 800) & (identified_clouds.alt <= 1100), drop=True
)

distance_IC_DS = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(f"data/observation/combined/distance/distance_dropsondes_clouds_{mask_name}.nc")
)

cloud_composite = xr.open_dataset(
    REPOSITORY_ROOT / Path("data/observation/cloud_composite/processed/cloud_composite.nc"),
    chunks={"time": 1000},
)

drop_sondes = xr.open_dataset(
    REPOSITORY_ROOT
    / Path("data/observation/dropsonde/Level_3/EUREC4A_JOANNE_Dropsonde-RD41_Level_3_v2.0.0.nc")
)
drop_sondes = drop_sondes.rename({"launch_time": "time"})
drop_sondes = drop_sondes.swap_dims({"sonde_id": "time"})
drop_sondes = drop_sondes.sortby("time")
drop_sondes = drop_sondes.chunk({"time": -1})

### Use Total number concentration

Chose an individual cloud to handle.
Use ``chosen_id = 77`` for the rain_mask case
Use ``chosen_id = 1421`` for the cloud_mask case

In [ ]:
# select a single cloud
if chosen_id is not None:
    ds_cloud = select_individual_cloud_by_id(identified_clouds, chosen_id)
else:
    ds_cloud = identified_clouds

ds_cloudcomposite = match_clouds_and_cloudcomposite(
    ds_clouds=ds_cloud,
    ds_cloudcomposite=cloud_composite,
)


# Make sure to have the total number of particles in the cloud See also #28 on GitHub
attrs = ds_cloudcomposite["particle_size_distribution"].attrs
attrs.update(
    {
        "unit": "#/L",
        "comment": "histogram: each bin gives the number of droplets per liter of air, NOT normalized by the bin width",
    }
)
ds_cloudcomposite["particle_size_distribution"] = (
    ds_cloudcomposite["particle_size_distribution"] * ds_cloudcomposite["bin_width"]
)
ds_cloudcomposite["particle_size_distribution"].attrs = attrs

ds_cloudcomposite

#### Plot the distributions in linear and lognormal space

In [ ]:
style = dict(
    marker=".",
    linestyle="none",
    color="k",
    alpha=0.5,
)
psd = ds_cloudcomposite["particle_size_distribution"]
symlog = symlog_from_array(psd)

fig, axss = plt.subplots(3, 2, figsize=(10, 7), layout="constrained")


fig.suptitle(f"Cloud ID: {chosen_id} - Particle Size Distribution - Different xscales")

for axs in axss.T:
    axs[0].plot(
        psd["diameter"],
        psd,
        **style,
    )
    # axs[0].set_title("Linear of x")
    axs[0].set_xlabel("Diameter [µm]")
    axs[0].set_ylabel("Counts [#/l]")
    axs[1].plot(
        psd["diameter"],
        psd,
        **style,
    )
    axs[1].set_xscale("log")
    # axs[1].set_title("Linear of x on log10 scale")
    axs[1].set_xlabel("Diameter [µm]")
    axs[1].set_ylabel("Counts [#/l]")

    axs[2].plot(
        np.log(psd["diameter"]),
        psd,
        **style,
    )
    # axs[2].set_title("Linear of Ln(x)")
    axs[2].set_xlabel("Ln(Diameter) [Ln(µm)]")
    axs[2].set_ylabel("Counts [#/l]")

for axs in axss.T[1]:
    axs.set_yscale(symlog)

for ax in axss.flatten():
    ax.set_ylabel("#/l")
    ax.set_ylim(0, None)

fig.savefig(subfig_path / Path(f"all_scales_psd_{mask_name}_cloud_{chosen_id}.png"), dpi=300)
fig.savefig(subfig_path / Path(f"all_scales_psd_{mask_name}_cloud_{chosen_id}.svg"))

# Ideas on how to fit a normal distribution

#### Use scipy curve fitting 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit

https://github.com/nilsnevertree/sdm-eurec4a/blob/c990ed160365230515fb583505a0514630339ef4/src/sdm_eurec4a/pySD/probdists.py#L224

# Combination of fitted cloud droplet distribution and aerosol distribution

## Make sure we use the same units

At the moment we have
- PSD in $\# l^{-1}$
- Diameter in $µm$

We want
- PSD in $\# m^{-3}$
- Radius in $m$
- Total number concentration $N_a$ in $\#$ (this is also the ``scalefacs``)

So the PSD is in #/dm^3, we want it in #/m^3

In [ ]:
ds_SI_Units = ds_cloudcomposite.copy()
# Convert from #/l to #/m^3 -> 1e3
ds_SI_Units["particle_size_distribution"] = ds_cloudcomposite["particle_size_distribution"] * 1e3
ds_SI_Units["particle_size_distribution"].attrs.update(
    unit="#/m^3",
    comment="histogram: each bin gives the number of droplets per cubic meter of air, NOT normalized by the bin width",
)
# Convert from µm to m -> 1e-6
ds_SI_Units["radius"] = ds_SI_Units["diameter"] / 2 * 1e-6
ds_SI_Units["radius"].attrs.update(long_name="Radius", unit="m", comment="radius of the droplets")
# Use radius as new dimension
ds_SI_Units = ds_SI_Units.swap_dims({"diameter": "radius"})
# display(ds_SI_Units)

Same plot as before but in SI units

In [ ]:
style = dict(
    marker=".",
    linestyle="none",
    color="k",
    alpha=0.5,
)
psd = ds_SI_Units["particle_size_distribution"]
symlog = symlog_from_array(psd)

fig, axss = plt.subplots(3, 2, figsize=(10, 7), layout="constrained")


fig.suptitle(f"Cloud ID: {chosen_id} - Particle Size Distribution - Different xscales")

for axs in axss.T:
    axs[0].plot(
        psd["radius"],
        psd,
        **style,
    )
    # axs[0].set_title("Linear of x")
    axs[0].set_xlabel("Radius [m]")
    axs[0].set_ylabel("Counts [#/m^3]")
    axs[1].plot(
        psd["radius"],
        psd,
        **style,
    )
    axs[1].set_xscale("log")
    # axs[1].set_title("Linear of x on log10 scale")
    axs[1].set_xlabel("Radius [m]")
    axs[1].set_ylabel("Counts [#/m^3]")

    axs[2].plot(
        np.log(psd["radius"]),
        psd,
        **style,
    )
    # axs[2].set_title("Linear of Ln(x)")
    axs[2].set_xlabel("Ln(Radius) [Ln(m)]")
    axs[2].set_ylabel("Counts [#/m^3]")

for axs in axss.T[1]:
    axs.set_yscale(symlog)

for ax in axss.flatten():
    ax.set_ylabel("#/m^3")
    ax.set_ylim(0, None)

fig.savefig(subfig_path / Path(f"SI_all_scales_psd_{mask_name}_cloud_{chosen_id}.png"), dpi=300)
fig.savefig(subfig_path / Path(f"SI_all_scales_psd_{mask_name}_cloud_{chosen_id}.svg"))

lets get some total number of droplets per timestep to get $N_a$ as in 5.2 from Lohmann et al.

The values of the PSD are NOT normalized by the bin width, thus we do NOT need to multiply by them again!

We can use the median of the $N_a$ as a scaling factor later during the fitting.

# Split data into cloud and rain

In [ ]:
def create_LnNormal_from_ds(
    ds: xr.Dataset,
    initial_guess: list,
    total_number_concentration: float,
    psd_name: str = "particle_size_distribution",
    radius_name: str = "radius",
    use_sigma: bool = True,
    **kwargs,
) -> probdists.LnNormal:
    """
    Create a LnNormal distribution from a dataset.

    Parameters
    ----------
    ds : xr.Dataset
        Dataset containing the PSD and radius.
    initial_guess : list
        Initial guess for the parameters
        - scale factor (total count of droplets),
        - geometrical mean,
        - geometrical standard deviation.
    total_number_concentration : float
        Total number of droplets in the cloud.
        If the cloud consists of multiple timesteps, use the median.
    psd_name : str, optional
        Name of the PSD variable, by default "particle_size_distribution"
    radius_name : str, optional
        Name of the radius variable, by default "radius"
    use_sigma : bool, optional
        Use sigma as uncertainty, by default True
    **kwargs
        Additional keyword arguments for the fit_parameters function.

    Returns
    -------
    probdists.LnNormal
        The fitted distribution.
    """

    xdata, ydata = x_y_flatten(ds[psd_name], radius_name)

    # make sure no nans in the dataset
    np.nan_to_num(xdata, copy=False, nan=0)
    np.nan_to_num(ydata, copy=False, nan=0)

    if use_sigma:
        # Use some default uncertainties for the data.
        # Tell the function, that 0 values are very uncertain.
        sigma = ydata == 0
        sigma = sigma.astype(float)
        sigma = sigma * 1e5 + 1e-18
    else:
        sigma = np.ones_like(ydata) * 1e5

    # Fit the parameters
    # initialize the cloud distribution using 1.0
    dist_cloud = probdists.LnNormal(
        geomeans=[1e0],
        geosigs=[1e0],
        scalefacs=[1e0],
    )

    # Fit the parameters
    dist_cloud.fit_parameters(xdata, ydata, p0=initial_guess, sigma=sigma, **kwargs)
    # Make sure to set the scaling factor of the cloud distribution to the total number of particles.
    # Here we use the median of the total number of particles from the ATR measurments.
    dist_cloud.scalefacs = [total_number_concentration]
    return dist_cloud

In [ ]:
split_radius = 4e-5

ds_cloud = ds_SI_Units.sel(radius=slice(None, split_radius))
ds_rain = ds_SI_Units.sel(radius=slice(split_radius, None))

if chosen_id is None:
    ds_cloud = ds_cloud.median(dim="time")
    ds_rain = ds_rain.median(dim="time")

# calculate the total number of droplets in the cloud
# calculate total number concentration
ds_cloud["N_a"] = ds_cloud["particle_size_distribution"].sum(dim="radius")
ds_cloud["N_a"].attrs.update(
    long_name="N_a total number of particles",
    unit="#",
    comment="total number of particles per cubic meter of air",
)

# calculate the total number of droplets in the rain
# calculate total number concentration
ds_rain["N_a"] = ds_rain["particle_size_distribution"].sum(dim="radius")
ds_rain["N_a"].attrs.update(
    long_name="N_a total number of particles",
    unit="#",
    comment="total number of particles per cubic meter of air",
)

### Cloud 

In [ ]:
if chosen_id is None:
    N_a_median_cloud = ds_cloud["N_a"].data
else:
    N_a_median_cloud = ds_cloud["N_a"].median("time").data

dist_cloud = create_LnNormal_from_ds(
    ds_cloud,
    initial_guess=[N_a_median_cloud, 2e-5, 1.38e00],
    total_number_concentration=N_a_median_cloud,
    use_sigma=True,
)

/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/pySD/probdists.py:187: RuntimeWarning: invalid value encountered in log
  mutilda = np.log(geomean)


In [ ]:
radii = np.logspace(-8, -3, 100)

# Compare the original data to the fit
fig, ax = plt.subplots(figsize=(5, 3.5), layout="constrained")
ax.plot(ds_SI_Units.radius, ds_SI_Units["particle_size_distribution"], "k.", alpha=0.5)
ax.plot(radii, dist_cloud(radii) * np.sum(dist_cloud.scalefacs), label="Fitted distribution")
ax.set_xscale("log")
ax.set_yscale(symlog)
ax.set_ylim(0, None)
ax.legend()
ax.set_xlabel("Radius [m]")
ax.set_ylabel("Counts [#/m3]")
ax.set_title("Cloud PSD and fitted distribution")

Text(0.5, 1.0, 'Cloud PSD and fitted distribution')

### Cloud 

In [ ]:
N_a_median_rain = ds_rain["N_a"].median("time").data

dist_rain = create_LnNormal_from_ds(
    ds_rain,
    initial_guess=[N_a_median_rain, 2e-5, 1.38e00],
    total_number_concentration=N_a_median_rain,
    use_sigma=True,
)
print(dist_rain)
dist_rain.scalefacs[0] = dist_rain.scalefacs[0] * 2e1
print(dist_rain)

nmodes = 1.00e+00
geomeans = [6.25e-05, ]
geosigs = [8.86e+00, ]
scalefacs = [2.67e+02, ]
numconc = 2.67e+02
nmodes = 1.00e+00
geomeans = [6.25e-05, ]
geosigs = [8.86e+00, ]
scalefacs = [5.35e+03, ]
numconc = 5.35e+03


In [ ]:
radii = np.logspace(-8, -3, 100)

# Compare the original data to the fit
fig, ax = plt.subplots(figsize=(5, 3.5), layout="constrained")
ax.plot(ds_SI_Units.radius, ds_SI_Units["particle_size_distribution"], "k.", alpha=0.5)
ax.plot(radii, dist_rain(radii) * np.sum(dist_rain.scalefacs), label="Fitted distribution")
ax.set_xscale("log")
ax.set_yscale(symlog)
ax.set_ylim(0, None)
ax.legend()
ax.set_xlabel("Radius [m]")
ax.set_ylabel("Counts [#/m3]")
ax.set_title("Cloud PSD and fitted distribution")

Text(0.5, 1.0, 'Cloud PSD and fitted distribution')

Give a first estimate of the fitting

### Important for correct plotting
Multiply by the scaling factor. It ``scalefacs`` is an array the use its sum! 

In [ ]:
radii = np.logspace(-8, -3, 100)

# Same Aerosol distribution as given by CLEO example
dist_aerosol = probdists.LnNormal(
    geomeans=[0.02e-6, 0.2e-6],
    geosigs=[1.55, 2.3],
    scalefacs=[1e9, 0.3e9],
)

# COMBINE THE DISTRIBUTIONS
dist_combined = (dist_aerosol + dist_cloud) + dist_rain

# Create dataset

ds_combined = xr.Dataset(
    data_vars={
        "particle_size_distribution": ("radius", dist_combined(radii) * np.sum(dist_combined.scalefacs)),
    },
    coords={"radius": radii},
    attrs={
        "name": "atr",
    },
)

ds_cloud = xr.Dataset(
    data_vars={
        "particle_size_distribution": ("radius", dist_cloud(radii) * np.sum(dist_cloud.scalefacs)),
    },
    coords={"radius": radii},
    attrs={
        "name": "atr",
    },
)

ds_aerosol = xr.Dataset(
    data_vars={
        "particle_size_distribution": ("radius", dist_aerosol(radii) * np.sum(dist_aerosol.scalefacs)),
    },
    coords={"radius": radii},
    attrs={
        "name": "atr",
    },
)


def add_msd(ds):
    ds["mass_size_distribution"] = msd_from_psd(
        ds=ds,
        psd_name="particle_size_distribution",
        psd_factor=1e6,
        scale_name="radius",
        radius_given=True,
        scale_factor=1e0,
    )


add_msd(ds_SI_Units)
add_msd(ds_combined)
add_msd(ds_cloud)
add_msd(ds_aerosol)

/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/conversions.py:69: UserWarning: units is set to m^3/m^3. Make sure to check the units and otherwise set the value!
  warnings.warn("units is set to m^3/m^3. Make sure to check the units and otherwise set the value!")
/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/conversions.py:146: UserWarning: units is set to kg/m^3. Make sure to check the units and otherwise set the value!
  warnings.warn("units is set to kg/m^3. Make sure to check the units and otherwise set the value!")


In [ ]:
style = dict(linewidth=3.5, alpha=0.7)


fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(7, 7), layout="constrained", sharex=True)
ax_psd = axs[0]
ax_msd = axs[1]

for ax in axs.flatten():
    ax.set_xscale("log")
    ax.set_xlabel("Radius [m]")


ax_psd.plot(
    ds_SI_Units.radius,
    ds_SI_Units["particle_size_distribution"],
    linestyle="none",
    marker=".",
    color="k",
    alpha=0.5,
)
ax_psd.plot(ds_aerosol.radius, ds_aerosol["particle_size_distribution"], label=f"Aersol PSD", **style)
ax_psd.plot(ds_cloud.radius, ds_cloud["particle_size_distribution"], label=f"Cloud PSD", **style)
ax_psd.plot(
    ds_combined.radius,
    ds_combined["particle_size_distribution"],
    label=f"Combined PSD",
    linestyle="--",
    color="g",
    **style,
)

ax_msd.plot(
    ds_SI_Units.radius,
    ds_SI_Units["mass_size_distribution"],
    linestyle="none",
    marker=".",
    color="k",
    alpha=0.5,
)
ax_msd.plot(ds_aerosol.radius, ds_aerosol["mass_size_distribution"], label=f"Aersol PSD", **style)
ax_msd.plot(ds_cloud.radius, ds_cloud["mass_size_distribution"], label=f"Cloud PSD", **style)

ax_msd.plot(
    ds_combined.radius,
    ds_combined["mass_size_distribution"],
    label=f"Combined PSD",
    linestyle="--",
    color="g",
    **style,
)

ax_psd.set_ylabel("Counts [#/m3]")
ax_msd.set_ylabel("Mass [kg/m3]")
ax_psd.set_title("Particle Size Distribution")
ax_msd.set_title("Mass Size Distribution")

for ax in axs.flatten():
    ax.legend()
    ax.set_yscale(symlog)

fig.suptitle(
    f"Aerosol distribution as in Lohmann et al. 2016 (Fig. 5.5) \n ATR measurment from Cloud {chosen_id}"
)
fig.savefig(subfig_path / Path(f"psd_msd_cloud_and_aerosol.png"), dpi=300)
fig.savefig(subfig_path / Path(f"psd_msd_cloud_and_aerosol.svg"))

In [ ]:
subfig_path

PosixPath('/home/m/m301096/repositories/sdm-eurec4a/results/CLEO/initilization/fitting_psd/rain_mask_77')

In [ ]:
print(dist_combined)

nmodes = 4.00e+00
geomeans = [2.00e-08, 2.00e-07, 6.87e-06, 6.25e-05, ]
geosigs = [1.55e+00, 2.30e+00, 1.05e+00, 8.86e+00, ]
scalefacs = [1.00e+09, 3.00e+08, 1.29e+08, 5.35e+03, ]
numconc = 1.43e+09


In [ ]:
geomeans = [
    3.77e-06,
]
geosigs = [
    1.38e00,
]
scalefacs = [
    2.73e08,
]
numconc = 2.73e08

In [ ]:
print(dist_rain)

nmodes = 1.00e+00
geomeans = [6.25e-05, ]
geosigs = [8.86e+00, ]
scalefacs = [5.35e+03, ]
numconc = 5.35e+03


In [ ]:
dist_cloud_1421 = probdists.LnNormal(
    geomeans=[
        3.77e-06,
    ],
    geosigs=[
        1.38e00,
    ],
    scalefacs=[
        2.73e08,
    ],
)
dist_better_cloud_77 = dist_cloud_1421 + dist_rain
print(dist_better_cloud_77)

ds_better = xr.Dataset(
    data_vars={
        "particle_size_distribution": (
            "radius",
            dist_better_cloud_77(radii) * np.sum(dist_better_cloud_77.scalefacs),
        ),
    },
    coords={"radius": radii},
    attrs={
        "name": "atr",
    },
)
ds_cloud_1421 = xr.Dataset(
    data_vars={
        "particle_size_distribution": (
            "radius",
            dist_cloud_1421(radii) * np.sum(dist_cloud_1421.scalefacs),
        ),
    },
    coords={"radius": radii},
    attrs={
        "name": "atr",
    },
)

ds_rain_fit = xr.Dataset(
    data_vars={
        "particle_size_distribution": ("radius", dist_rain(radii) * np.sum(dist_rain.scalefacs)),
    },
    coords={"radius": radii},
    attrs={
        "name": "atr",
    },
)

add_msd(ds_better)
add_msd(ds_cloud_1421)
add_msd(ds_rain_fit)

nmodes = 2.00e+00
geomeans = [3.77e-06, 6.25e-05, ]
geosigs = [1.38e+00, 8.86e+00, ]
scalefacs = [2.73e+08, 5.35e+03, ]
numconc = 2.73e+08


In [ ]:
style = dict(linewidth=3.5, alpha=0.7)


fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(7, 7), layout="constrained", sharex=True)
ax_psd = axs[0]
ax_msd = axs[1]

for ax in axs.flatten():
    ax.set_xscale("log")
    ax.set_xlabel("Radius [m]")


ax_psd.plot(
    ds_SI_Units.radius,
    ds_SI_Units["particle_size_distribution"],
    linestyle="none",
    marker=".",
    color="k",
    alpha=0.5,
)
ax_psd.plot(ds_rain_fit.radius, ds_rain_fit["particle_size_distribution"], label=f"Rain PSD", **style)
ax_psd.plot(
    ds_cloud_1421.radius,
    ds_cloud_1421["particle_size_distribution"],
    label=f"Cloud fit from 1421 PSD",
    **style,
)

ax_psd.plot(
    ds_better.radius,
    ds_better["particle_size_distribution"],
    label=f"Combined PSD",
    linestyle="--",
    color="g",
    **style,
)

ax_msd.plot(
    ds_SI_Units.radius,
    ds_SI_Units["mass_size_distribution"],
    linestyle="none",
    marker=".",
    color="k",
    alpha=0.5,
)
ax_msd.plot(ds_rain_fit.radius, ds_rain_fit["mass_size_distribution"], label=f"Rain MSD", **style)
ax_msd.plot(ds_cloud_1421.radius, ds_cloud_1421["mass_size_distribution"], label=f"Cloud MSD", **style)

ax_msd.plot(
    ds_better.radius,
    ds_better["mass_size_distribution"],
    label=f"Combined MSD",
    linestyle="--",
    color="g",
    **style,
)

ax_psd.set_ylabel("Counts [#/m3]")
ax_msd.set_ylabel("Mass [kg/m3]")
ax_psd.set_title("Particle Size Distribution")
ax_msd.set_title("Mass Size Distribution")

for ax in axs.flatten():
    ax.legend()
    ax.set_yscale(symlog)

fig.suptitle(f"ATR measurment from Cloud {chosen_id}")
fig.savefig(subfig_path / Path(f"1421and77_psd_msd_cloud_and_aerosol.png"), dpi=300)
fig.savefig(subfig_path / Path(f"1421and77_psd_msd_cloud_and_aerosol.svg"))

In [ ]:
print(dist_better_cloud_77)

nmodes = 2.00e+00
geomeans = [3.77e-06, 6.25e-05, ]
geosigs = [1.38e+00, 8.86e+00, ]
scalefacs = [2.73e+08, 5.35e+03, ]
numconc = 2.73e+08


In [ ]:
msd_from_psd(ds_rain, "particle_size_distribution").sum(dim="radius", keep_attrs=True).sum(
    dim="time", keep_attrs=True
)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.5), layout="constrained")

symlog = symlog_from_array(ds_rain["particle_size_distribution"], offset=0, linthresh=4e1)

ax.set_xscale("log")
ax.set_yscale(symlog)
ax.plot(
    ds_rain.radius,
    ds_rain["particle_size_distribution"],
    label=f"Rain PSD",
    marker=".",
    linestyle="none",
    alpha=0.5,
)
ax.plot(ds_rain_fit.radius, ds_rain_fit["particle_size_distribution"], label=f"Rain fit PSD", **style)